## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-04.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.003997,-0.002842,-0.004031,-0.002391,0.001861,0.005982,0.004598,-0.007833,-0.009146,-0.003277,...,0.003287,-0.001843,0.001108,0.022880,0.019664,-0.000855,0.004881,-0.005456,0.000501,0.004790
2,-0.007872,-0.012111,0.003190,-0.004176,0.003429,0.033265,0.031457,-0.006351,0.066918,0.016314,...,-0.003943,-0.005541,0.001686,0.037975,0.004156,0.077593,0.024541,-0.004940,0.006880,0.021386


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.000000,-0.002842,-0.004031,-0.002391,0.0,0.0,0.0,-0.007833,-0.009146,-0.003277,...,0.000000,-0.001843,0.0,0.0,0.0,-0.000855,0.0,-0.005456,0.0,0.0
2,-0.007872,-0.012111,0.000000,-0.004176,0.0,0.0,0.0,-0.006351,0.000000,0.000000,...,-0.003943,-0.005541,0.0,0.0,0.0,0.000000,0.0,-0.004940,0.0,0.0


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.017611,0.013752,-0.010714,0.002649,-0.002327,-0.040481,-0.039851,-0.002199,-0.112855,-0.029066,...,0.010727,0.005486,-0.000857,-0.022396,0.023009,-0.116391,-0.029170,-0.000765,-0.009464,-0.024624
BAJFINANCE,0.013752,0.010740,-0.008367,0.002069,-0.001817,-0.031612,-0.031120,-0.001717,-0.088131,-0.022699,...,0.008377,0.004284,-0.000669,-0.017490,0.017968,-0.090893,-0.022779,-0.000597,-0.007391,-0.019229
BAJAJFINSV,-0.010714,-0.008367,0.006518,-0.001612,0.001416,0.024627,0.024244,0.001338,0.068658,0.017683,...,-0.006526,-0.003337,0.000521,0.013625,-0.013998,0.070810,0.017746,0.000465,0.005758,0.014981
BRITANNIA,0.002649,0.002069,-0.001612,0.000399,-0.000350,-0.006089,-0.005995,-0.000331,-0.016977,-0.004372,...,0.001614,0.000825,-0.000129,-0.003369,0.003461,-0.017509,-0.004388,-0.000115,-0.001424,-0.003704
DIVISLAB,-0.002327,-0.001817,0.001416,-0.000350,0.000307,0.005349,0.005266,0.000291,0.014912,0.003841,...,-0.001417,-0.000725,0.000113,0.002959,-0.003040,0.015379,0.003854,0.000101,0.001251,0.003254
GRASIM,-0.040481,-0.031612,0.024627,-0.006089,0.005349,0.093051,0.091604,0.005054,0.259415,0.066814,...,-0.024658,-0.012610,0.001969,0.051481,-0.052890,0.267544,0.067051,0.001758,0.021755,0.056602
HCLTECH,-0.039851,-0.031120,0.024244,-0.005995,0.005266,0.091604,0.090179,0.004975,0.255380,0.065774,...,-0.024274,-0.012414,0.001939,0.050681,-0.052067,0.263382,0.066008,0.001731,0.021416,0.055721
HDFCBANK,-0.002199,-0.001717,0.001338,-0.000331,0.000291,0.005054,0.004975,0.000274,0.014089,0.003629,...,-0.001339,-0.000685,0.000107,0.002796,-0.002873,0.014531,0.003642,0.000095,0.001182,0.003074
HINDALCO,-0.112855,-0.088131,0.068658,-0.016977,0.014912,0.259415,0.255380,0.014089,0.723218,0.186268,...,-0.068744,-0.035155,0.005490,0.143524,-0.147451,0.745881,0.186930,0.004901,0.060650,0.157799
HINDUNILVR,-0.029066,-0.022699,0.017683,-0.004372,0.003841,0.066814,0.065774,0.003629,0.186268,0.047974,...,-0.017705,-0.009054,0.001414,0.036965,-0.037977,0.192105,0.048145,0.001262,0.015621,0.040642


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.592299,0.098377,15.82175,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array(BALANCE([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
]))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,1.592299,0.009788,159.013191,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')